In [ ]:
# Change directory to the root so that relative path loads work correctly
import os

try:
    os.chdir(os.path.join(os.getcwd(), ".."))
    print(os.getcwd())
except:
    pass

In [ ]:
import glob
import sys

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import torch

from experiments.C_nonlinear_projection.main import build_model_and_optimizer, get_data
from experiments.C_nonlinear_projection.visualize import (  # plot_model_predictions,
    plot_epoch_wise,
    plot_epoch_wise_distribution,
    plot_model_predictions,
    plot_time,
    retrieve_object,
)

In [ ]:
def convert_name_to_filename(model_name):
    filename = model_name.replace(" ", "-").lower()
    filename = (
        filename.replace("(", "").replace(")", "").replace(":", "").replace(",", "")
    )
    return filename

In [ ]:
def get_model_name(checkpoint):
    config = checkpoint["configuration"]
    weight = config["regularization_weight"]
    weight_string = (
        "Unconstrained" if weight == 0.0 else f"Soft Constrained ({weight:g})"
    )
    #     model_act = config["model_act"]
    epoch = checkpoint["epoch"]
    return f"{weight_string} Epoch {epoch}"


def get_group_name(checkpoint):
    config = checkpoint["configuration"]
    weight = config["regularization_weight"]
    weight_string = (
        "Unconstrained" if weight == 0.0 else f"Soft Constrained ({weight:g})"
    )
    return weight_string

In [ ]:
def get_special_model_name(checkpoint, filename):
    if "nonlinear-projection_2019-08-20-11-32-32_00010.pth" in filename:
        return "Huber Error"
    elif "nonlinear-projection_2019-08-20-11-35-45_00010.pth" in filename:
        return "Mean Squared Error"
    else:
        return get_model_name(checkpoint)

In [ ]:
# Files to load
experiment_name = "C_nonlinear_projection"
save_directory = f"/global/u1/g/gelijerg/Projects/pyinsulate/results/{experiment_name}/"
load_directory = os.path.expandvars(f"$SCRATCH/results/checkpoints/{experiment_name}")
checkpoint_patterns = [
    "nonlinear-projection_2019-08-20-11-32-32_00010.pth",  # Huber Error
    "nonlinear-projection_2019-08-20-11-35-45_00010.pth",  # Mean Squared Error
]

In [ ]:
# Load files
files = list()
for pattern in checkpoint_patterns:
    files.extend(glob.glob(f"{load_directory}/{pattern}"))
files.sort()
print(files)
checkpoints = [torch.load(f, map_location=torch.device("cpu")) for f in files]
# model_names = [get_model_name(checkpoint) for checkpoint in checkpoints]
model_names = [
    get_special_model_name(checkpoint, filename)
    for checkpoint, filename in zip(checkpoints, files)
]
# Make sure directory to save exists
os.makedirs(save_directory, exist_ok=True)

In [ ]:
# Do some plotting
max_epoch = max([checkpoint["epoch"] for checkpoint in checkpoints])
final_checkpoints = [
    (checkpoint, model_name)
    for (checkpoint, model_name) in zip(checkpoints, model_names)
    if checkpoint["epoch"] == max_epoch
]

tasks = [("Final Models", final_checkpoints)]

checkpoint_groups = dict()
for f, checkpoint, model_name in zip(files, checkpoints, model_names):
    key = os.path.basename(f[: f.rfind("_")])
    if key not in checkpoint_groups:
        checkpoint_groups[key] = list()
    checkpoint_groups[key].append((checkpoint, model_name))
for group_key, checkpoint_name_pairs in checkpoint_groups.items():
    name = get_group_name(checkpoint_name_pairs[0][0])
    tasks.append((name, checkpoint_name_pairs))

for task_name, task in tasks:
    print(task_name)
    if len(task) == 0:
        print(f"Nothing for task {task_name}")
        continue
    task_checkpoints = [x[0] for x in task]
    task_model_names = [x[1] for x in task]
    task_monitors = [checkpoint["monitors"] for checkpoint in task_checkpoints]
    task_predictions = [checkpoint["predictions"] for checkpoint in task_checkpoints]
    task_filename = convert_name_to_filename(task_name)

    if "final" in task_filename:
        # TRAINING
        fig = plot_time(
            task_monitors,
            task_model_names,
            f"{task_filename}_time",
            log=True,
            directory=save_directory,
        )
        fig = plot_epoch_wise(
            [monitors[0] for monitors in task_monitors],
            task_model_names,
            [retrieve_object(monitors[0], "total_loss") for monitors in task_monitors],
            f"{task_filename}_training-total-loss",
            title="Training Total Loss",
            ylabel="Average loss",
            log=True,
            directory=save_directory,
        )
        fig = plot_epoch_wise(
            [monitors[0] for monitors in task_monitors],
            task_model_names,
            [retrieve_object(monitors[0], "mean_loss") for monitors in task_monitors],
            f"{task_filename}_training-loss",
            title="Training Data Loss",
            ylabel="Average loss",
            log=True,
            directory=save_directory,
        )
        fig = plot_epoch_wise(
            [monitors[0] for monitors in task_monitors],
            task_model_names,
            [
                retrieve_object(monitors[0], "constraints_error")
                for monitors in task_monitors
            ],
            f"{task_filename}_training-constraints-error",
            title="Training Constraint Error",
            ylabel="Average constraint error",
            log=True,
            directory=save_directory,
        )
        fig = plot_epoch_wise_distribution(
            [monitors[0] for monitors in task_monitors],
            task_model_names,
            [
                retrieve_object(monitors[0], "constraints", absolute_value=False)
                for monitors in task_monitors
            ],
            f"{task_filename}_training-constraint-distribution",
            title="Training Distribution of Constraint Residual",
            ylabel="Constraint value",
            log=False,
            directory=save_directory,
        )
        fig = plot_epoch_wise_distribution(
            [monitors[0] for monitors in task_monitors],
            task_model_names,
            [
                retrieve_object(monitors[0], "constraints", absolute_value=True)
                for monitors in task_monitors
            ],
            f"{task_filename}_training-constraint-distribution-magnitude",
            title="Training Distribution of Magnitude of Constraint Residual",
            ylabel="Magnitude of constraint value",
            log=True,
            directory=save_directory,
        )

        # Inference
        monitors_to_plot = list()
        names_to_plot = list()
        data_to_plot = list()
        colors = list()
        line_styles = list()
        for i, (monitors, name) in enumerate(zip(task_monitors, task_model_names)):
            monitors_to_plot.extend([monitors[2], monitors[2]])
            names_to_plot.extend([f"{name} (Unprojected)", f"{name} (Projected)"])
            data_to_plot.extend(
                [
                    [x[0] for x in retrieve_object(monitors[2], "mean_loss")],
                    [x[-1] for x in retrieve_object(monitors[2], "mean_loss")],
                ]
            )
            colors.extend([i, i])
            line_styles.extend(["--", "-"])
        fig = plot_epoch_wise(
            monitors_to_plot,
            names_to_plot,
            data_to_plot,
            f"{task_filename}_testing-loss",
            colors=colors,
            line_styles=line_styles,
            title="Inference Data Loss",
            ylabel="Average loss",
            log=True,
            directory=save_directory,
        )

        monitors_to_plot = list()
        names_to_plot = list()
        data_to_plot = list()
        colors = list()
        line_styles = list()
        for i, (monitors, name) in enumerate(zip(task_monitors, task_model_names)):
            monitors_to_plot.extend([monitors[2], monitors[2]])
            names_to_plot.extend([f"{name} (Unprojected)", f"{name} (Projected)"])
            data_to_plot.extend(
                [
                    [x[0] for x in retrieve_object(monitors[2], "constraints_error")],
                    [x[-1] for x in retrieve_object(monitors[2], "constraints_error")],
                ]
            )
            colors.extend([i, i])
            line_styles.extend(["--", "-"])
        fig = plot_epoch_wise(
            monitors_to_plot,
            names_to_plot,
            data_to_plot,
            f"{task_filename}_testing-constraints-error",
            colors=colors,
            line_styles=line_styles,
            title="Inference Constraint Error",
            ylabel="Average constraint error",
            log=True,
            directory=save_directory,
        )
        fig = plot_epoch_wise(
            [monitors[2] for monitors in task_monitors],
            task_model_names,
            [
                retrieve_object(monitors[2], "projection_epochs")
                for monitors in task_monitors
            ],
            f"{task_filename}_projection_epochs",
            title="Projection Epochs",
            ylabel="Number of epochs",
            log=False,
            directory=save_directory,
        )

        # Maybe put the constraint distributions here, but I think it might be too much
        inputs = None
        outputs = None
        prediction_sets = list()
        colors = list()
        line_styles = list()
        for i, (predictions, name) in enumerate(
            zip(task_predictions, task_model_names)
        ):
            # Assumption: all models have the same test set
            if inputs is None:
                inputs = predictions.inputs
            if outputs is None:
                outputs = predictions.outputs
            prediction_sets.extend(
                [predictions.predictions[-1][0], predictions.predictions[-1][-1]]
            )
            names_to_plot.extend([f"{name} (Unprojected)", f"{name} (Projected)"])
            colors.extend([i, i])
            line_styles.extend(["--", "-"])
        fig = plot_model_predictions(
            inputs,
            outputs,
            prediction_sets,
            names_to_plot,
            f"{task_filename}_predictions",
            colors=colors,
            line_styles=line_styles,
            title="Model Predictions",
            directory=save_directory,
        )

        # Model-wise
        for monitors, predictions, model_name in zip(
            task_monitors, task_predictions, task_model_names
        ):
            model_filename = convert_name_to_filename(model_name)

            fig = plot_epoch_wise_distribution(
                [monitors[0]],
                [model_name],
                [retrieve_object(monitors[0], "model_parameters", gradients=False)],
                f"{task_filename}_{model_filename}_parameter-distribution",
                title=f"Distribution of Parameter Values",
                ylabel="Parameter values",
                log=False,
                directory=save_directory,
            )
            fig = plot_epoch_wise_distribution(
                [monitors[0]],
                [model_name],
                [retrieve_object(monitors[0], "model_parameters", gradients=True)],
                f"{task_filename}_{model_filename}_gradient-distribution",
                title=f"Distribution of Parameter Gradients",
                ylabel="Parameter gradients",
                log=False,
                directory=save_directory,
            )
            fig = plot_epoch_wise_distribution(
                [monitors[2]],
                [model_name],
                [retrieve_object(monitors[2], "model_parameters", differences=True)],
                f"{task_filename}_{model_filename}_parameter-differences",
                title=f"Distribution of Parameter Differences From Projection",
                ylabel="Parameter differences",
                log=False,
                directory=save_directory,
            )
            # Plot process of projection
            inputs = predictions.inputs
            outputs = predictions.outputs
            prediction_sets = [predictions.predictions[-1][0]]
            names_to_plot = ["Unprojected"]
            colors = [0]
            line_styles = ["--"]
            for i in range(1, len(predictions.predictions[-1]) - 1):
                prediction_sets.append(predictions.predictions[-1][i])
                names_to_plot.append(None)
                line_styles.append("--")
                colors.append((0.5, 0.5, 0.5, 0.2))
            prediction_sets.append(predictions.predictions[-1][-1])
            names_to_plot.append("Projected")
            line_styles.append("-")
            colors.append(0)  # Same color as original
            fig = plot_model_predictions(
                inputs,
                outputs,
                prediction_sets,
                names_to_plot,
                f"{task_filename}_{model_filename}_projections",
                colors=colors,
                line_styles=line_styles,
                title=f"{model_name} Model Predictions",
                directory=save_directory,
            )
            # Plot predictions over training
            cmap = mpl.cm.get_cmap("rainbow")
            num_epochs = len(monitors[2].projection_epochs)
            prediction_sets = list()
            names_to_plot = list()
            raw_colors = [cmap(x) for x in np.linspace(0, 1, num=num_epochs)]
            colors = list()
            line_styles = list()
            for i in range(num_epochs):
                prediction_sets.extend(
                    [predictions.predictions[i][0], predictions.predictions[i][-1]]
                )
                if i == num_epochs - 1:
                    names_to_plot.extend(["Unprojected", "Projected"])
                else:
                    names_to_plot.extend([None, None])
                colors.extend([raw_colors[i], raw_colors[i]])
                line_styles.extend(["--", "-"])
            fig = plot_model_predictions(
                inputs,
                outputs,
                prediction_sets,
                names_to_plot,
                f"{task_filename}_{model_filename}_predictions",
                colors=colors,
                line_styles=line_styles,
                title=f"{model_name} Model Predictions",
                directory=save_directory,
                cmap=cmap,
                cbar_endpoints=(1, num_epochs),
            )

    else:  # this is probably a checkpoint group for a single run

        print(f"Nothing implemented for {task_name}")